In [1]:
import numpy as np
from numpy import sqrt, sin, cos, pi, exp
import matplotlib.pyplot as plt
from scipy.integrate import dblquad, quad
import timeit

In [ ]:
"""Question 1"""

## super Gaussian form
G = lambda x,y: exp( -((x**2 + y**2)/w**2)**n )

"a"
y = 0
w = 1e-3
lim = 3e-3
x = np.linspace(-lim,lim,1000)

plt.figure(1)
n = 1
plt.plot(x,G(x,y),label = '$n=1$')
n = 3
plt.plot(x,G(x,y), label = '$n=3$')
n = 10
plt.plot(x,G(x,y), label = '$n=10$')
plt.xlabel('$x$')
plt.ylabel('Intensity')
plt.title('Gaussian Form $G(x,y) =  e^{ -(\\frac{x^2 + y^2}{w^2})^n } $')
plt.legend()

"b"

y = x.copy()
[x,y] = np.meshgrid(x,y)
fig = plt.figure(2)
ax = fig.add_subplot(111)
CB = ax.pcolor(x, y, G(x,y))
fig.colorbar(CB, ax=ax)
plt.xlabel('$x$')
plt.ylabel('$y$')
plt.title('Gaussian Form $G(x,y) =  e^{ -(\\frac{x^2 + y^2}{w^2})^n } $')

In [ ]:
"""Question 2"""

## Create function of Huygen Fresnel Integrand

def H(x,y,x_p,y_p,z,n,w):
    # The Huygen Frensel integrand for λ = 532e-9
    # x = x
    # y = y
    # x_p = x'
    # y_p = y'
    # z = beam propagation distance
    # n = mode?
    # w = beam radius?

    λ = 532e-9
    k = 2*pi/λ

    r = sqrt( (x - x_p)**2 + (y - y_p)**2 + z**2 )
    G = exp( -((x_p**2 + y_p**2)/w**2)**n )

    integrand = G*(z*exp(1j*k*r))/r

    return integrand



In [ ]:
"""Question 3"""

## Plot Real & Imaginary part of integrand for two values of z

lim = 2e-3
points = 1000

z = [1/2,3]
w = 1.5e-3
n = 10

x = np.zeros((points))
x_p = np.linspace(-lim,lim,points)
y = np.zeros((points))
y_p = y.copy()

plt.subplot(211)
plt.plot(x_p,H(x,y,x_p,y_p,z[0],n,w).real, label = '$z=0.5m$')
plt.plot(x_p,H(x,y,x_p,y_p,z[1],n,w).real, label = '$z=3m$')
plt.ylabel('$Real$')
plt.title('Huygen Frensel Integrand, $ \\frac{z}{r} e^{ ikr -(\\frac{x^2 + y^2}{w^2})^n } $')
plt.legend(bbox_to_anchor=(1.05,1), loc = 'upper left')
plt.subplot(212)
plt.plot(x_p,H(x,y,x_p,y_p,z[0],n,w).imag, label = '$z=0.5m$')
plt.plot(x_p,H(x,y,x_p,y_p,z[1],n,w).imag, label = '$z=3m$')
plt.ylabel('$Imaginary$')
plt.xlabel('$x^{\prime}$')


In [ ]:
"""Question 4"""

## Compute integral for a single point on the outputplace, x=y=0, z = 1

lim = 2*w
z = 1
λ = 532e-9

# x = np.zeros((2,points))
#x[1] = np.linspace(-lim,lim,points)
# y = np.zeros((2,points))
#y[1] = np.linspace(-lim,lim,points)

H_r = lambda y_p,x_p: H(0,0,x_p,y_p,z,n,w).real
H_i = lambda y_p,x_p: H(0,0,x_p,y_p,z,n,w).imag

int_r = dblquad(H_r,-lim,lim,-lim,lim)
int_i = dblquad(H_i,-lim,lim,-lim,lim)

print(f'Frensel Diffraction Integral: {1j/λ *(int_r[0] + int_i[0])} metres.')

In [ ]:
"""Question 5"""

## use timeit to measure the duration of integration for z = [1,1/4]m

z = [1,1/4]

# for z = 1
H_r = lambda y_p,x_p: H(0,0,x_p,y_p,z[0],n,w).real
H_i = lambda y_p,x_p: H(0,0,x_p,y_p,z[0],n,w).imag
time_z0 = %timeit -n 1 -r 1 int_r = (dblquad(H_r,-lim,lim,-lim,lim) + dblquad(H_i,-lim,lim,-lim,lim)) 


In [ ]:
# for z = 1/4

H_r = lambda y_p,x_p: H(0,0,x_p,y_p,z[1],n,w).real
H_i = lambda y_p,x_p: H(0,0,x_p,y_p,z[1],n,w).imag
time_z1 = %timeit -n 1 -r 1 int_r = (dblquad(H_r,-lim,lim,-lim,lim) + dblquad(H_i,-lim,lim,-lim,lim))

Question 5

The time elapsed for the operations of summing the integrations via dblquad at $z = 1$ was 163ms. For $z=\frac{1}{4}$ the time was 724ms. In terms of the integral itself, as $z$ approaches the same dimensions of the given limits $ x',y' \in [-2w,2w], w = 1.5mm $, the resolution of the integration increases. More 'non-zero' values are computed as the integration range is now well defined.

Also recalling the Frensel Number:
$$
N_f = \frac{a^2}{z\lambda}
$$

As $z \rightarrow \infty \implies N_f \rightarrow 0$, removing effects of near field diffraction. For $N_F \approx 1$ the product of $\lambda w$ must equal the apperature size $a^2$, which is assumed to be in the micrometers or less. 

Hence, the calculation of the Huygens Frensel diffraction integral becomes more intense as $z \rightarrow \frac{a^2}{\lambda}$, because this improves the system resolution.

In [ ]:
"""Question 6"""

## use a for-loop to calculate absolute value of the integral over a 1D slice through the output 
## x = [-3,3]mm, set z = fixed value
## this means that integrate volume over x,y,z


n = 50
x = np.linspace(-3e-3,3e-3,n)
z = 1

total = 0
int_r = np.zeros(n)
int_i = np.zeros(n)

for i in range(0,n):
    # integrating over dy, with x being in the range given.
    # iterate over each x[i] value

    H_r = lambda y: H(x[i],y,0,0,z,n,w).real
    H_i = lambda y: H(x[i],y,0,0,z,n,w).imag

    int_r[i] = quad(H_r,-3e-3,3e-3)[0]
    int_i[i] = quad(H_i,-3e-3,3e-3)[0]

    total = total + abs(int_r[i]) + abs(int_i[i])
    
# clearly something isn't right here because I can do 100 datapoints in less than 10s

plt.plot(x,int_r, label = 'Real')
plt.plot(x,int_i, label = 'Imaginary')
plt.xlabel('x')
plt.ylabel('Integral')
plt.legend()

print(f'Absolute value of the integral {total}')